# Extração das Imagens

In [32]:
import glob
from shutil import copyfile

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
participants = glob.glob("base\\source_emotion\\*")

for x in participants:
    part = "%s" % x[-4:]
    for sessions in glob.glob("%s\\*" % x):
        for files in glob.glob("%s\\*" % sessions):
            current_session = files[25:-30]

            file = open(files, 'r')

            emotion = int(float(file.readline()))

            sourcefile_emotion = glob.glob("base\source_images\\%s\\%s\\*" % (part, current_session))[-1]
            sourcefile_neutral = glob.glob("base\source_images\\%s\\%s\\*" % (part, current_session))[0]

            dest_neut = "base\sorted_set\\neutral\\%s" % sourcefile_neutral[28:]
            dest_emot = "base\sorted_set\\%s\\%s" % (emotions[emotion], sourcefile_emotion[28:])

            copyfile(sourcefile_neutral, dest_neut)
            copyfile(sourcefile_emotion, dest_emot)
            
print("Extação Concluida!")

Extação Concluida!


# Detecção das Faces

In [34]:
import cv2
import glob

faceDet = cv2.CascadeClassifier("C:\\opencv\\build\\etc\\haarcascades\\haarcascade_frontalface_default.xml")
faceDet_two = cv2.CascadeClassifier("C:\\opencv\\build\\etc\\haarcascades\\haarcascade_frontalface_alt2.xml")
faceDet_three = cv2.CascadeClassifier("C:\\opencv\\build\\etc\\haarcascades\\haarcascade_frontalface_alt.xml")
faceDet_four = cv2.CascadeClassifier("C:\\opencv\\build\\etc\\haarcascades\\haarcascade_frontalface_alt_tree.xml")

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]


def detect_faces(emotion):
    files = glob.glob("base\\sorted_set\\{}\\*".format(emotion))

    filenumber = 0
    i = 1
    for f in files:
        frame = cv2.imread(f)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_two = faceDet_two.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_three = faceDet_three.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_four = faceDet_four.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)

        if len(face) == 1:
            facefeatures = face
        elif len(face_two) == 1:
            facefeatures = face_two
        elif len(face_three) == 1:
            facefeatures = face_three
        elif len(face_four) == 1:
            facefeatures = face_four
        else:
            facefeatures = ""


        for (x, y, w, h,) in facefeatures:
            # print("Face Encontrada no arquivo {}: {}".format(i, f))

            gray = gray[y:y + h, x:x + w]

            try:
                out = cv2.resize(gray, (350, 350))
                cv2.imwrite("base\\dataset\\%s\\%s.jpg" % (emotion, filenumber), out)
            except:
                pass
        i = i + 1
        filenumber += 1


for emotion in emotions:
    detect_faces(emotion)
    print("Emoção {} concluída!".format(emotion))

Emoção neutral concluída!
Emoção anger concluída!
Emoção contempt concluída!
Emoção disgust concluída!
Emoção fear concluída!
Emoção happy concluída!
Emoção sadness concluída!
Emoção surprise concluída!


# DecisionTreeRegressor

In [19]:
import cv2
import glob
import random
import numpy as np
from sklearn import tree
from sklearn.metrics import accuracy_score

emotions = ["happy", "surprise"]

regr = tree.DecisionTreeRegressor(criterion='mae', splitter='best')

file = open('base\\resultados.txt', 'w')

def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.7)]
    test = files[-int(len(files) * 0.3):]
    return training, test


def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    return training_X, training_y, test_X, test_y

def run_recognizer(regr):
    training_X, training_y, test_X, test_y = make_sets()

    training_X = np.array(training_X)
    training_y = np.array(training_y)

    test_X = np.array(test_X)
    test_y = np.array(test_y)

    training_X = training_X.reshape(106, 350*350)
    test_X = test_X.reshape(44, 350*350)

    regr = regr.fit(training_X, training_y)

    predict = regr.predict(test_X)

    return accuracy_score(predict, test_y)


metascore = []

for i in range(0, 3):
    correct = run_recognizer(regr)
    file.write("{} {}\n".format(i, correct))
    #print("{}".format(i))
    metascore.append(correct)

metascore = np.array(metascore)

print("Accuracy: %1.2f%%" % (metascore.mean()*100))
print("Margin of Error for More and Less: %1.2f%%" % (metascore.std()*100))

file.write("{}\n".format(metascore.mean()))
file.write("{}\n".format(metascore.std()))

file.close()

Acuracy: 84.85%
Margin of Error for More and Less: 5.67%



# RandomForest


In [37]:
import cv2
import glob
import random
import numpy as np
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
# emotions = ["happy", "surprise"]

clf = RandomForestClassifier(n_estimators=500)

clf = RandomForestClassifier(n_estimators=500, max_features=.3)

file = open('base\\resultados.txt', 'w')

def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.7)]
    test = files[-int(len(files) * 0.3):]
    return training, test


def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    return training_X, training_y, test_X, test_y

def run_recognizer(clf):
    training_X, training_y, test_X, test_y = make_sets()

    training_X = np.array(training_X)
    training_y = np.array(training_y)

    test_X = np.array(test_X)
    test_y = np.array(test_y)

    training_X = training_X.reshape(training_X.shape[0], 350*350)
    test_X = test_X.reshape(test_X.shape[0], 350*350)

    clf = clf.fit(training_X, training_y)

    predict = clf.predict(test_X)

    return accuracy_score(predict, test_y)


metascore = []

for i in range(0, 3):
    correct = run_recognizer(clf)
    file.write("{} {}\n".format(i, correct))
    # print("{}".format(i))
    metascore.append(correct)

metascore = np.array(metascore)

print("Accuracy: %1.2f%%" % (metascore.mean()*100))
print("Margin of Error for More and Less: %1.2f%%" % (metascore.std()*100))

file.write("{}\n".format(metascore.mean()))
file.write("{}\n".format(metascore.std()))

file.close()

Accuracy: 70.47%
Margin of Error for More and Less: 0.73%


# ExtraTreesClassifier

In [3]:
import cv2
import glob
import random
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier

metascore = []

# emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
emotions = ["happy", "surprise"]

clf = ExtraTreesClassifier(n_estimators=500)

def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.8)]
    test = files[-int(len(files) * 0.2):]
    return training, test


def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    training_X = np.array(training_X)
    training_y = np.array(training_y)
    test_X = np.array(test_X)
    test_y = np.array(test_y)

    return training_X, training_y, test_X, test_y


def run_recognizer(clf):
    training_X, training_y, test_X, test_y = make_sets()

    training_X = training_X.reshape(training_X.shape[0], 350*350)
    test_X = test_X.reshape(test_X.shape[0], 350*350)

    clf = clf.fit(training_X, training_y)

    predict = clf.predict(test_X)

    score = clf.score(test_X, test_y)

    cm = confusion_matrix(test_y, predict, labels=[0, 1, 2, 3, 4, 5, 6, 7])

    accuracy = accuracy_score(predict, test_y)

    return accuracy, score, cm


for i in range(0, 3):

    correct, score, cm = run_recognizer(clf)

    print("Etapa {}".format(i+1))

    print("Score: {}".format(score))

    print("Matriz de confusao:")
    print(cm)

    metascore.append(correct)

metascore = np.array(metascore)

print("Acuracy: %1.2f%%" % (metascore.mean()*100))
print("Margin of Error for More and Less: %1.2f%%" % (metascore.std()*100))


Etapa 1
Score: 0.7265625
Matriz de confusao:
[[59  3  1  1  0  0  1  0]
 [ 7  2  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0]
 [ 5  0  0  7  0  0  0  0]
 [ 3  1  0  1  0  0  0  0]
 [ 1  0  0  0  0 12  0  0]
 [ 4  1  0  0  0  0  0  0]
 [ 2  1  0  0  0  0  0 13]]


KeyboardInterrupt: 

# Confusion Matrix

In [ ]:
import cv2
import glob
import random
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

metascore = []

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
# emotions = ["happy", "surprise"]

clf = RandomForestClassifier(n_estimators=500, max_features=.3)

def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.8)]
    test = files[-int(len(files) * 0.2):]
    return training, test


def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    training_X = np.array(training_X)
    training_y = np.array(training_y)
    test_X = np.array(test_X)
    test_y = np.array(test_y)

    return training_X, training_y, test_X, test_y


def run_recognizer(clf):
    training_X, training_y, test_X, test_y = make_sets()

    training_X = training_X.reshape(training_X.shape[0], 350*350)
    test_X = test_X.reshape(test_X.shape[0], 350*350)

    clf = clf.fit(training_X, training_y)

    predict = clf.predict(test_X)

    score = clf.score(test_X, test_y)

    cm = confusion_matrix(test_y, predict, labels=[0, 1, 2, 3, 4, 5, 6, 7])

    accuracy = accuracy_score(predict, test_y)

    return accuracy, score, cm


for i in range(0, 3):

    correct, score, cm = run_recognizer(clf)

    print("Etapa {}".format(i+1))

    print("Score: {}".format(score))

    print("Matriz de confusao:")
    print(cm)

    metascore.append(correct)

metascore = np.array(metascore)

print("Acuracy: %1.2f%%" % (metascore.mean()*100))
print("Margin of Error for More and Less: %1.2f%%" % (metascore.std()*100))


Etapa 1
Score: 0.7265625
Matriz de confusao:
[[65  0  0  0  0  0  0  0]
 [ 8  0  0  0  0  1  0  0]
 [ 3  0  0  0  0  0  0  0]
 [ 7  1  0  4  0  0  0  0]
 [ 4  0  0  0  0  0  0  1]
 [ 2  0  0  0  0 11  0  0]
 [ 5  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0 13]]


# GridSearch

In [7]:
# -*- coding: utf-8 -*-
import cv2
import glob
import random
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

metascore = []

# emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
emotions = ["happy", "surprise"]

clf = RandomForestClassifier(max_features=500)

def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.8)]
    test = files[-int(len(files) * 0.2):]
    return training, test


def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    training_X = np.array(training_X)
    training_y = np.array(training_y)
    test_X = np.array(test_X)
    test_y = np.array(test_y)

    return training_X, training_y, test_X, test_y


def run_recognizer(clf):
    training_X, training_y, test_X, test_y = make_sets()

    training_X = training_X.reshape(training_X.shape[0], 350*350)
    test_X = test_X.reshape(test_X.shape[0], 350*350)

    clf = clf.fit(training_X, training_y)

    predict = clf.predict(test_X)

    score = clf.score(test_X, test_y)

    cm = confusion_matrix(test_y, predict, labels=[0, 1, 2, 3, 4, 5, 6, 7])

    accuracy = accuracy_score(predict, test_y)

    param_grid = {
        'n_estimators': [200, 500, 700],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': [4, 5, 6, 7, 8],
        'criterion': ['gini', 'entropy'],
        'min_samples_split': [10, 20, 30, 40, 50]
    }

    grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
    grid.fit(training_X, training_y)

    y_true, y_pred = test_y, grid.predict(test_X)

    bs = grid.best_score_
    bp = grid.best_params_
    cr = classification_report(y_true, y_pred)  # isso também mostra o precision, recall e f1-score


    return accuracy, score, cm, bs, bp, cr


for i in range(0, 3):

    correct, score, cm, bs, bp, cr = run_recognizer(clf)

    print("Etapa {}".format(i+1))

    print("Score: {}".format(score))

    print("\nMatriz de confusao:")
    print(cm)

    print("\nBest Score:")
    print(bs)

    print("\nBest Params:")
    print(bp)

    print("\nClassification Report:")
    print(cr)

    metascore.append(correct)

metascore = np.array(metascore)

print("\nAcuracy: %1.2f%%" % (metascore.mean()*100))
print("Margin of Error for More and Less: %1.2f%%" % (metascore.std()*100))


Etapa 1
Score: 0.9655172413793104

Matriz de confusao:
[[13  0  0  0  0  0  0  0]
 [ 1 15  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]

Best Score:
0.99173553719

Best Params:
{'criterion': 'gini', 'max_depth': 4, 'max_features': 'auto', 'min_samples_split': 20, 'n_estimators': 200}

Classification Report:
             precision    recall  f1-score   support

          0       0.93      1.00      0.96        13
          1       1.00      0.94      0.97        16

avg / total       0.97      0.97      0.97        29

Etapa 2
Score: 1.0

Matriz de confusao:
[[13  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]

Best Score:
0.99173553719

Best Params:
{'criterion': 'gini', 'max_depth': 4, 'max

# RandomSearch

In [9]:
# -*- coding: utf-8 -*-
import cv2
import glob
import random
import numpy as np
from scipy.stats import randint as sp_randint
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

metascore = []

# emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
emotions = ["happy", "surprise"]

clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)


def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.8)]
    test = files[-int(len(files) * 0.2):]
    return training, test


def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    training_X = np.array(training_X)
    training_y = np.array(training_y)
    test_X = np.array(test_X)
    test_y = np.array(test_y)

    return training_X, training_y, test_X, test_y


def run_recognizer(clf):
    training_X, training_y, test_X, test_y = make_sets()

    training_X = training_X.reshape(training_X.shape[0], 350 * 350)
    test_X = test_X.reshape(test_X.shape[0], 350 * 350)

    clf = clf.fit(training_X, training_y)

    predict = clf.predict(test_X)

    score = clf.score(test_X, test_y)

    cm = confusion_matrix(test_y, predict, labels=[0, 1, 2, 3, 4, 5, 6, 7])

    accuracy = accuracy_score(predict, test_y)

    param_dist = {"max_depth" :  [3, 4, 5, 6, 7],
                  "max_features" : sp_randint(2, 100),
                  "min_samples_split" : sp_randint(8, 30),
                  "min_samples_leaf" : sp_randint(5, 20),
                  "n_estimators" : [100, 200, 300, 400, 500, 600],
                  "bootstrap" : [True, False],
                  "criterion" : ["gini", "entropy"]
                  }

    grid = RandomizedSearchCV(estimator=clf, param_distributions = param_dist, n_iter=100)
    grid.fit(training_X, training_y)

    y_true, y_pred = test_y, grid.predict(test_X)

    bs = grid.best_score_
    bp = grid.best_params_
    cr = classification_report(y_true, y_pred)  # isso também mostra o precision, recall e f1-score

    return accuracy, score, cm, bs, bp, cr


for i in range(0, 3):
    correct, score, cm, bs, bp, cr = run_recognizer(clf)

    print('\033[1m Etapa: ' + '\033[0m{}\n'.format(i + 1))

    print ('\033[1m Score: ' + '\033[1;34m {} \n'.format(score) + '\033[0m')

    print('\033[1m Matriz de confusao:\n\n' + '\033[0m {} \n'.format(cm))

    print('\033[1m Best Score: ' + '\033[1;34m {}\n'.format(bs) + '\033[0m')

    print('\033[1m Best Params:\n\n' + '\033[0m {}\n'.format(bp))

    print('\033[1m Classification Report:\n\n' + '\033[0m {}'.format(cr))

    metascore.append(correct)

metascore = np.array(metascore)

print("\n\033[1m Acuracy: \033[1;34m %1.2f%% \033[0m" % (metascore.mean() * 100))
print("\033[1m Margin of Error for More and Less: \033[1;34m %1.2f%%" % (metascore.std() * 100))


 Etapa: 1

 Score:  1.0 

 Matriz de confusao:

 [[13  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]] 

 Best Score:  0.9834710743801653

 Best Params:

 {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 6, 'max_features': 51, 'min_samples_leaf': 5, 'min_samples_split': 16, 'n_estimators': 200}

 Classification Report:

              precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       1.00      1.00      1.00        16

avg / total       1.00      1.00      1.00        29

 Etapa: 2

 Score:  0.9310344827586207 

 Matriz de confusao:

 [[13  0  0  0  0  0  0  0]
 [ 2 14  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]] 

 Best S

# GradientBoosting

In [15]:
# -*- coding: utf-8 -*-
import cv2
import glob
import random
import numpy as np
from scipy.stats import randint as sp_randint
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

metascore = []

# emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
emotions = ["happy", "surprise"]

clf = GradientBoostingClassifier(n_estimators=500)


def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.8)]
    test = files[-int(len(files) * 0.2):]
    return training, test


def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    training_X = np.array(training_X)
    training_y = np.array(training_y)
    test_X = np.array(test_X)
    test_y = np.array(test_y)

    return training_X, training_y, test_X, test_y


def run_recognizer(clf):
    training_X, training_y, test_X, test_y = make_sets()

    training_X = training_X.reshape(training_X.shape[0], 350 * 350)
    test_X = test_X.reshape(test_X.shape[0], 350 * 350)

    clf = clf.fit(training_X, training_y)

    predict = clf.predict(test_X)

    score = clf.score(test_X, test_y)

    cm = confusion_matrix(test_y, predict, labels=[0, 1, 2, 3, 4, 5, 6, 7])

    accuracy = accuracy_score(predict, test_y)

    param_dist = {"max_depth" :  [3, 4, 5, 6, 7],
                  "max_features" : sp_randint(2, 100),
                  "min_samples_split" : sp_randint(8, 30),
                  "min_samples_leaf" : sp_randint(5, 20),
                  "n_estimators" : [100, 200, 300, 400, 500, 600],
                  "criterion" : ["mae", "mse"]
                  }

    grid = RandomizedSearchCV(estimator=clf, param_distributions = param_dist, n_iter=100)
    grid.fit(training_X, training_y)

    y_true, y_pred = test_y, grid.predict(test_X)

    bs = grid.best_score_
    bp = grid.best_params_
    cr = classification_report(y_true, y_pred)  # isso também mostra o precision, recall e f1-score

    return accuracy, score, cm, bs, bp, cr


for i in range(0, 3):
    correct, score, cm, bs, bp, cr = run_recognizer(clf)

    print('\033[1m Etapa: ' + '\033[0m{}\n'.format(i + 1))

    print ('\033[1m Score: ' + '\033[1;34m {} \n'.format(score) + '\033[0m')

    print('\033[1m Matriz de confusao:\n\n' + '\033[0m {} \n'.format(cm))

    print('\033[1m Best Score: ' + '\033[1;34m {}\n'.format(bs) + '\033[0m')

    print('\033[1m Best Params:\n\n' + '\033[0m {}\n'.format(bp))

    print('\033[1m Classification Report:\n\n' + '\033[0m {}'.format(cr))

    metascore.append(correct)

metascore = np.array(metascore)

print("\n\033[1m Acuracy: \033[1;34m %1.2f%% \033[0m" % (metascore.mean() * 100))
print("\033[1m Margin of Error for More and Less: \033[1;34m %1.2f%%" % (metascore.std() * 100))


 Etapa: 1

 Score:  0.896551724137931 

 Matriz de confusao:

 [[11  2  0  0  0  0  0  0]
 [ 1 15  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]] 

 Best Score:  0.9834710743801653

 Best Params:

 {'criterion': 'mse', 'max_depth': 3, 'max_features': 52, 'min_samples_leaf': 6, 'min_samples_split': 28, 'n_estimators': 600}

 Classification Report:

              precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       1.00      1.00      1.00        16

avg / total       1.00      1.00      1.00        29

 Etapa: 2

 Score:  0.896551724137931 

 Matriz de confusao:

 [[12  1  0  0  0  0  0  0]
 [ 2 14  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]] 

 Best Score:  0.99

# PCA em um Pipeline

In [3]:
# -*- coding: utf-8 -*-
import cv2
import glob
import random
import numpy as np
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from scipy.stats import randint as sp_randint
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

metascore = []

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
# emotions = ["happy", "surprise"]


def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.8)]
    test = files[-int(len(files) * 0.2):]
    return training, test


def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    training_X = np.array(training_X)
    training_y = np.array(training_y)
    test_X = np.array(test_X)
    test_y = np.array(test_y)

    return training_X, training_y, test_X, test_y


def run_recognizer(clf):
    training_X, training_y, test_X, test_y = make_sets()

    training_X = training_X.reshape(training_X.shape[0], 350 * 350)
    test_X = test_X.reshape(test_X.shape[0], 350 * 350)

    clf = clf.fit(training_X, training_y)

    predict = clf.predict(test_X)

    score = clf.score(test_X, test_y)

    cm = confusion_matrix(test_y, predict, labels=[0, 1, 2, 3, 4, 5, 6, 7])

    accuracy = accuracy_score(predict, test_y)

    # print(clf.get_params().keys())

    param_dist = {"randomforestclassifier__max_depth": [5, 6, 7],
                  "randomforestclassifier__max_features": [.1, .25, .5, .75, 1. ],
                  "randomforestclassifier__n_estimators": [400, 500, 600],
                  "randomforestclassifier__bootstrap" :  [True, False],
                  "randomforestclassifier__n_jobs" : [50, 100, 200],
                  "randomforestclassifier__criterion": ["gini", "entropy"],
                  "pca__svd_solver" : ["full","randomized"],
                  "pca__n_components" : [200, 500, 700]
                  }

    grid = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=100)
    grid.fit(training_X, training_y)

    y_true, y_pred = test_y, grid.predict(test_X)

    bs = grid.best_score_
    bp = grid.best_params_
    cr = classification_report(y_true, y_pred)  # isso também mostra o precision, recall e f1-score

    return accuracy, score, cm, bs, bp, cr

clf = make_pipeline(PCA(copy=True,
                        iterated_power='auto',
                        n_components=2000,
                        random_state=sp_randint(2, 20),
                        svd_solver='full',
                        whiten=False
                        ),
                    RandomForestClassifier(bootstrap=False,
                                           criterion='entropy',
                                           max_depth=6,
                                           max_features=51,
                                           min_samples_leaf=5,
                                           min_samples_split=16,
                                           n_estimators=200
                                          )
                    )


# GradientBoostingClassifier(n_estimators=500)
# ExtraTreesClassifier(n_estimators=500)


for i in range(0, 3):
    correct, score, cm, bs, bp, cr = run_recognizer(clf)

    print('\033[1m Etapa: ' + '\033[0m{}\n'.format(i + 1))

    print ('\033[1m Score: ' + '\033[1;34m {} \n'.format(score) + '\033[0m')

    print('\033[1m Matriz de confusao:\n\n' + '\033[0m {} \n'.format(cm))

    print('\033[1m Best Score: ' + '\033[1;34m {}\n'.format(bs) + '\033[0m')

    print('\033[1m Best Params:\n\n' + '\033[0m {}\n'.format(bp))

    print('\033[1m Classification Report:\n\n' + '\033[0m {}'.format(cr))

    metascore.append(correct)

metascore = np.array(metascore)

print("\n\033[1m Acuracy: \033[1;34m %1.2f%% \033[0m" % (metascore.mean() * 100))
print("\033[1m Margin of Error for More and Less: \033[1;34m %1.2f%%" % (metascore.std() * 100))


KeyboardInterrupt: 

# Voting

In [ ]:
# -*- coding: utf-8 -*-
import cv2
import glob
import random
import numpy as np
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from scipy.stats import randint as sp_randint
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

metascore1 = []
metascore2 = []
metascore3 = []
metascore4 = []

# emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
emotions = ["happy", "surprise"]


def get_param_dist(label):
    if (label == 'Random Forest'):
        param_dist = {"randomforestclassifier__max_depth": [3, 4, 5, 6, 7],
                      "randomforestclassifier__max_features": [.1, .25, .5, .75, 1.],
                      "randomforestclassifier__max_leaf_nodes": sp_randint(2, 100),
                      "randomforestclassifier__min_samples_split": sp_randint(8, 30),
                      "randomforestclassifier__min_samples_leaf": sp_randint(5, 20),
                      "randomforestclassifier__n_estimators": [100, 200, 300, 400, 500, 600],
                      "randomforestclassifier__bootstrap": [True, False],
                      "randomforestclassifier__n_jobs": [5, 20, 50, 100, 200],
                      "randomforestclassifier__criterion": ["gini", "entropy"],
                      "pca__iterated_power": ["auto"],
                      "pca__svd_solver": ["full", "randomized"],
                      "pca__whiten": [True, False],
                      "pca__random_state": sp_randint(2, 20),
                      "pca__n_components": [200, 500, 700, 1000, 2000]
                      }
    elif(label == 'Extra Trees'):

        param_dist = {"extratreesclassifier__n_estimators": [100, 200, 300, 400, 500, 600],
                      "pca__whiten": [True, False],
                      "extratreesclassifier__max_features": [.1, .25, .5, .75, 1.],
                      "extratreesclassifier__min_samples_split": sp_randint(8, 30),
                      "pca__iterated_power": ["auto"],
                      "pca__random_state": sp_randint(2, 20),
                      "pca__svd_solver": ["full", "randomized"],
                      "extratreesclassifier__bootstrap": [True, False],
                      "extratreesclassifier__n_jobs": [5, 20, 50, 100, 200],
                      "extratreesclassifier__max_leaf_nodes": sp_randint(2, 100),
                      "extratreesclassifier__criterion": ["gini", "entropy"],
                      "pca__n_components": [200, 500, 700, 1000, 2000],
                      "extratreesclassifier__min_samples_leaf": sp_randint(5, 20),
                      "extratreesclassifier__max_depth": [3, 4, 5, 6, 7]
                      }
    elif(label == 'Gradient Boosting'):

        param_dist = {"pca__whiten": [True, False],
                      "gradientboostingclassifier__min_samples_leaf": sp_randint(5, 20),
                      "gradientboostingclassifier__criterion": [],
                      "gradientboostingclassifier__max_leaf_nodes": sp_randint(2, 100),
                      "pca__iterated_power": ["auto"],
                      "gradientboostingclassifier__max_features": [.1, .25, .5, .75, 1.],
                      "gradientboostingclassifier__n_estimators": [100, 200, 300, 400, 500, 600],
                      "pca__svd_solver": ["full", "randomized"],
                      "pca__n_components": [200, 500, 700, 1000, 2000],
                      "gradientboostingclassifier__max_depth": [3, 4, 5, 6, 7]
                      }
    else:
        param_dist = {}

    return param_dist

def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.8)]
    test = files[-int(len(files) * 0.2):]
    return training, test


def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    training_X = np.array(training_X)
    training_y = np.array(training_y)
    test_X = np.array(test_X)
    test_y = np.array(test_y)

    return training_X, training_y, test_X, test_y


def run_recognizer(clf, label):
    training_X, training_y, test_X, test_y = make_sets()

    training_X = training_X.reshape(training_X.shape[0], 350 * 350)
    test_X = test_X.reshape(test_X.shape[0], 350 * 350)

    clf = clf.fit(training_X, training_y)

    predict = clf.predict(test_X)

    score = clf.score(test_X, test_y)

    cm = confusion_matrix(test_y, predict, labels=[0, 1, 2, 3, 4, 5, 6, 7])

    accuracy = accuracy_score(predict, test_y)

    print(clf.get_params().keys())

    param_dist = get_param_dist(label)

    grid = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=100)
    grid.fit(training_X, training_y)

    y_true, y_pred = test_y, grid.predict(test_X)

    bs = grid.best_score_
    bp = grid.best_params_
    cr = classification_report(y_true, y_pred)  # isso também mostra o precision, recall e f1-score

    return accuracy, score, cm, bs, bp, cr


clf1 = make_pipeline(PCA(copy=True,
                         iterated_power='auto',
                         n_components=2000,
                         random_state=sp_randint(2, 20),
                         svd_solver='full',
                         whiten=False
                         ),
                     RandomForestClassifier(n_estimators=700, n_jobs=-1)
                     )

clf2 = make_pipeline(PCA(copy=True,
                         iterated_power='auto',
                         n_components=2000,
                         random_state=sp_randint(2, 20),
                         svd_solver='full',
                         whiten=False
                         ),
                     ExtraTreesClassifier(n_estimators=500)
                     )

clf3 = make_pipeline(PCA(copy=True,
                         iterated_power='auto',
                         n_components=2000,
                         random_state=sp_randint(2, 20),
                         svd_solver='full',
                         whiten=False
                         ),
                     GradientBoostingClassifier(n_estimators=500)
                     )

eclf = VotingClassifier(estimators=[('rf', clf1), ('et', clf2), ('gb', clf3)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Random Forest', 'Extra Trees', 'Gradient Boosting', 'Voting']):

    for i in range(0, 3):
        correct, score, cm, bs, bp, cr = run_recognizer(clf, label)

        print('\033[1m Etapa: ' + '\033[0m{}\n'.format(i + 1))

        print ('\033[1m Score: ' + '\033[1;34m {} \n'.format(score) + '\033[0m')

        print('\033[1m Matriz de confusao:\n\n' + '\033[0m {} \n'.format(cm))

        print('\033[1m Best Score: ' + '\033[1;34m {}\n'.format(bs) + '\033[0m')

        print('\033[1m Best Params:\n\n' + '\033[0m {}\n'.format(bp))

        print('\033[1m Classification Report:\n\n' + '\033[0m {}'.format(cr))

        if(label == 'Random Forest'):
            metascore1.append(correct)
        elif(label == 'Extra Trees'):
            metascore2.append(correct)
        elif(label == 'Gradient Boosting'):
            metascore3.append(correct)
        else:
            metascore4.append(correct)

metascore1 = np.array(metascore1)
metascore2 = np.array(metascore2)
metascore3 = np.array(metascore3)
metascore4 = np.array(metascore4)

print("\n\033[1m Acuracy the \033[1;31m Random Forest\033[1m: \033[1;34m %1.2f%% \033[0m" % (metascore1.mean() * 100))
print("\033[1m Margin of Error for More and Less: \033[1;34m %1.2f%% \n" % (metascore1.std() * 100))

print("\n\033[1m Acuracy the \033[1;31m Extra Trees\033[1m: \033[1;34m %1.2f%% \033[0m" % (metascore2.mean() * 100))
print("\033[1m Margin of Error for More and Less: \033[1;34m %1.2f%% \n" % (metascore2.std() * 100))

print("\n\033[1m Acuracy the \033[1;31m Gradient Boosting\033[1m: \033[1;34m %1.2f%% \033[0m" % (metascore3.mean() * 100))
print("\033[1m Margin of Error for More and Less: \033[1;34m %1.2f%% \n" % (metascore3.std() * 100))

print("\n\033[1m Acuracy the \033[1;31m Voting\033[1m: \033[1;34m %1.2f%% \033[0m" % (metascore4.mean() * 100))
print("\033[1m Margin of Error for More and Less: \033[1;34m %1.2f%% \n" % (metascore4.std() * 100))


dict_keys(['memory', 'steps', 'pca', 'randomforestclassifier', 'pca__copy', 'pca__iterated_power', 'pca__n_components', 'pca__random_state', 'pca__svd_solver', 'pca__tol', 'pca__whiten', 'randomforestclassifier__bootstrap', 'randomforestclassifier__class_weight', 'randomforestclassifier__criterion', 'randomforestclassifier__max_depth', 'randomforestclassifier__max_features', 'randomforestclassifier__max_leaf_nodes', 'randomforestclassifier__min_impurity_decrease', 'randomforestclassifier__min_impurity_split', 'randomforestclassifier__min_samples_leaf', 'randomforestclassifier__min_samples_split', 'randomforestclassifier__min_weight_fraction_leaf', 'randomforestclassifier__n_estimators', 'randomforestclassifier__n_jobs', 'randomforestclassifier__oob_score', 'randomforestclassifier__random_state', 'randomforestclassifier__verbose', 'randomforestclassifier__warm_start'])
